In [1]:
# Libraries
import time
from collections import deque, namedtuple # deque -> memory buffer, namedtuple -> store experience tuples

# FFMPEG - required for Video Wrappers
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/Cellar/ffmpeg/7.1_3/bin/ffmpeg"
print("FFmpeg Path:", os.environ.get("IMAGEIO_FFMPEG_EXE"))

# Import Gym Environment
import gym
from gym.wrappers import RecordVideo

import pandas
import numpy as np
import tensorflow as tf
import PIL.Image
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras

# Import Utility Functions
import utils

FFmpeg Path: /opt/homebrew/Cellar/ffmpeg/7.1_3/bin/ffmpeg


In [2]:
# Import Trained Kera Model
modelFile = 'lunar_lander_model.keras'
dqnModel = tf.keras.models.load_model(modelFile)

# Show the model architecture
dqnModel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ BN1 (BatchNormalization)        │ (None, 8)              │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN2 (BatchNormalization)        │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,284 (20.64 KB)

 Trainable params: 5,140 (20.08 KB)

 Non-trainable params: 144 (576.00 B)

In [3]:
# Trigger Recording for every episode number divisible by 2
def episodeTrigger(episode):
    return episode % 2 == 0

In [4]:
# Set up Open AI Gym Environment
env = gym.make("LunarLander-v2", render_mode = "rgb_array")
env = RecordVideo(env, video_folder="videos", episode_trigger = episodeTrigger)

In [5]:
# Create Loop of tests - 5 Episodes
for i in range(5):
    # receive initial observation state
    state, info = env.reset()
    for t in range(1000):
        # State (1,8) input matrix
        state_qn = np.expand_dims(state, axis = 0)
        q_vals = dqnModel(state_qn)
        # define action as max output of model - no epsilon
        action = np.argmax(q_vals.numpy()[0])
        next_state, reward, done, trunc, info = env.step(action)
        #break if terminal
        if done:
            break
        # move env to next state
        state = next_state.copy()

/opt/anaconda3/lib/python3.12/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


MoviePy - Building video /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-0.mp4.
MoviePy - Writing video /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-0.mp4



MoviePy - Done !
MoviePy - video ready /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-0.mp4


MoviePy - Building video /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-2.mp4.
MoviePy - Writing video /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-2.mp4



MoviePy - Done !
MoviePy - video ready /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-2.mp4


MoviePy - Building video /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-4.mp4.
MoviePy - Writing video /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-4.mp4



MoviePy - Done !
MoviePy - video ready /Users/nikhiltrivedi/Desktop/DQN/videos/rl-video-episode-4.mp4


In [6]:
env.close()